Sistema de recomendación

In [2]:
import pandas as pd
import numpy as np
import os
import json
import pickle
from datetime import datetime
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import warnings
warnings.filterwarnings('ignore')

# Crear carpeta para resultados si no existe
resultados_dir = "./resultados_recomendaciones"
if not os.path.exists(resultados_dir):
    os.makedirs(resultados_dir)
    print(f"✓ Carpeta creada: {resultados_dir}")
else:
    print(f"✓ Carpeta existente: {resultados_dir}")

# Cargar y examinar datos
data_dir = "./datos_procesados/"

# Cargar datasets
porcentajes = pd.read_csv(f"{data_dir}todos_porcentajes_procesado.csv")
ciclos = pd.read_csv(f"{data_dir}todos_ciclos_procesado.csv")

print(f"✓ Datos cargados:")
print(f"  - Porcentajes: {porcentajes.shape}")
print(f"  - Ciclos: {ciclos.shape}")

# Examinar la estructura de los datos
print("\nColumnas en porcentajes:")
print(porcentajes.columns.tolist())
print("\nPrimeras filas de porcentajes:")
print(porcentajes.head())

class SistemaRecomendacionAdaptativo:
    def __init__(self):
        self.scaler = StandardScaler()
        self.label_encoders = {}
        self.knn_model = None
        self.kmeans_model = None
        self.data_prepared = None
        self.feature_columns = None
        self.porcentaje_col = None
        self.available_columns = {}
        
    def detectar_columnas(self, df):
        """Detecta automáticamente las columnas disponibles y sus tipos"""
        self.available_columns = {
            'porcentaje': None,
            'familia': None,
            'sexo': None,
            'comunidad': None,
            'nivel': None
        }
        
        # Detectar columna de porcentaje
        for col in df.columns:
            if any(term in col.lower() for term in ['porcentaje', 'porcentage', '%', 'aprobados', 'resultado']):
                self.available_columns['porcentaje'] = col
                self.porcentaje_col = col
                break
        
        # Detectar otras columnas
        for col in df.columns:
            col_lower = col.lower()
            if 'familia' in col_lower and 'profesional' in col_lower:
                self.available_columns['familia'] = col
            elif 'sexo' in col_lower:
                self.available_columns['sexo'] = col
            elif 'comunidad' in col_lower or 'autónoma' in col_lower or 'autonoma' in col_lower:
                self.available_columns['comunidad'] = col
            elif 'nivel' in col_lower or 'educativo' in col_lower:
                self.available_columns['nivel'] = col
        
        print("\nColumnas detectadas:")
        for tipo, columna in self.available_columns.items():
            print(f"  {tipo}: {columna}")
        
        return self.available_columns
    
    def preparar_datos(self, porcentajes_df, ciclos_df):
        """Prepara los datos adaptándose a las columnas disponibles"""
        # Detectar columnas
        self.detectar_columnas(porcentajes_df)
        
        if self.porcentaje_col is None:
            raise ValueError("No se encontró una columna de porcentajes")
        
        # Copiar datos
        df = porcentajes_df.copy()
        
        # Limpiar datos
        df = df[df[self.porcentaje_col].notna()]
        
        # Filtrar totales si existe columna de familia
        if self.available_columns['familia']:
            df = df[df[self.available_columns['familia']] != 'TOTAL']
            df = df[df[self.available_columns['familia']].notna()]
        
        # Guardar información de ciclos
        self.ciclos_info = ciclos_df.copy()
        
        # Crear columna de éxito (porcentaje >= 80)
        df['exito'] = (df[self.porcentaje_col] >= 80).astype(int)
        
        # Preparar features disponibles
        features_categoricas = []
        for tipo, col in self.available_columns.items():
            if col and tipo != 'porcentaje' and col in df.columns:
                features_categoricas.append(col)
                # Crear encoder para esta columna
                self.label_encoders[col] = LabelEncoder()
                df[f'{col}_encoded'] = self.label_encoders[col].fit_transform(df[col].astype(str))
        
        print(f"\nFeatures categóricas encontradas: {features_categoricas}")
        
        # Crear columnas dummy si faltan features importantes
        if not features_categoricas:
            df['dummy'] = 'A'
            self.label_encoders['dummy'] = LabelEncoder()
            df['dummy_encoded'] = self.label_encoders['dummy'].fit_transform(df['dummy'])
            features_categoricas = ['dummy']
        
        # Seleccionar columnas de features
        self.feature_columns = [f'{col}_encoded' for col in features_categoricas]
        
        # Preparar X e y
        X = df[self.feature_columns].values
        y = df['exito'].values
        
        # Escalar features
        X_scaled = self.scaler.fit_transform(X)
        
        # Guardar datos preparados
        self.data_prepared = df
        
        print(f"\nDatos preparados: {len(df)} registros con {len(self.feature_columns)} features")
        print(f"Tasa de éxito: {y.mean():.2%}")
        
        return X_scaled, y
    
    def entrenar_modelos(self, X, y):
        """Entrena los modelos de recomendación"""
        print("\n=== Entrenando modelos ===")
        
        # Split datos
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42, stratify=y
        )
        
        # 1. Entrenar KNN
        print("Entrenando KNN...")
        self.knn_model = KNeighborsClassifier(n_neighbors=min(7, len(X_train)))
        self.knn_model.fit(X_train, y_train)
        
        y_pred_knn = self.knn_model.predict(X_test)
        accuracy_knn = accuracy_score(y_test, y_pred_knn)
        print(f"KNN Accuracy: {accuracy_knn:.3f}")
        
        # 2. Entrenar K-means
        print("\nEntrenando K-means...")
        n_clusters = min(5, len(np.unique(y)))
        self.kmeans_model = KMeans(n_clusters=n_clusters, random_state=42)
        self.kmeans_model.fit(X)
        
        # Asignar clusters a datos
        self.data_prepared['cluster'] = self.kmeans_model.labels_
        
        # Analizar clusters
        cluster_analysis = self.data_prepared.groupby('cluster').agg({
            self.porcentaje_col: ['mean', 'std', 'count'],
            'exito': 'mean'
        })
        
        print("\nAnálisis de clusters:")
        print(cluster_analysis)
        
        # Guardar modelos
        self._guardar_modelos()
        
        return accuracy_knn, cluster_analysis
    
    def _guardar_modelos(self):
        """Guarda los modelos entrenados"""
        modelos_dir = f"{resultados_dir}/modelos"
        if not os.path.exists(modelos_dir):
            os.makedirs(modelos_dir)
        
        # Guardar modelos y configuración
        with open(f"{modelos_dir}/knn_model.pkl", 'wb') as f:
            pickle.dump(self.knn_model, f)
        
        with open(f"{modelos_dir}/kmeans_model.pkl", 'wb') as f:
            pickle.dump(self.kmeans_model, f)
        
        with open(f"{modelos_dir}/scaler.pkl", 'wb') as f:
            pickle.dump(self.scaler, f)
        
        with open(f"{modelos_dir}/label_encoders.pkl", 'wb') as f:
            pickle.dump(self.label_encoders, f)
        
        # Guardar información de columnas
        with open(f"{modelos_dir}/column_mapping.json", 'w') as f:
            json.dump(self.available_columns, f)
        
        print("✓ Modelos guardados en:", modelos_dir)
    
    def crear_formulario_entrada(self):
        """Crea un formulario adaptativo basado en las columnas disponibles"""
        print("\n=== SISTEMA DE RECOMENDACIÓN DE FORMACIÓN PROFESIONAL ===")
        print("Por favor, introduce tu información para recibir recomendaciones personalizadas:\n")
        
        perfil_alumno = {}
        
        # Sexo - si está disponible
        if self.available_columns['sexo'] and self.available_columns['sexo'] in self.label_encoders:
            col_sexo = self.available_columns['sexo']
            sexos_disponibles = list(self.label_encoders[col_sexo].classes_)
            print("1. Sexo:")
            for i, sexo in enumerate(sexos_disponibles, 1):
                print(f"   {i}. {sexo}")
            try:
                sexo_idx = int(input(f"Selecciona una opción (1-{len(sexos_disponibles)}): ")) - 1
                perfil_alumno['sexo'] = sexos_disponibles[sexo_idx]
            except:
                perfil_alumno['sexo'] = sexos_disponibles[0]
        
        # Comunidad Autónoma - si está disponible
        if self.available_columns['comunidad'] and self.available_columns['comunidad'] in self.label_encoders:
            col_comunidad = self.available_columns['comunidad']
            print("\n2. Comunidad Autónoma:")
            comunidades = sorted(self.label_encoders[col_comunidad].classes_)
            # Mostrar solo las primeras 20 si hay muchas
            comunidades_mostrar = comunidades[:20] if len(comunidades) > 20 else comunidades
            for i, comunidad in enumerate(comunidades_mostrar, 1):
                print(f"   {i}. {comunidad}")
            if len(comunidades) > 20:
                print(f"   ... y {len(comunidades) - 20} más")
            try:
                comunidad_idx = int(input(f"Selecciona una opción (1-{len(comunidades_mostrar)}): ")) - 1
                perfil_alumno['comunidad'] = comunidades_mostrar[comunidad_idx]
            except:
                perfil_alumno['comunidad'] = comunidades[0]
        
        # Nivel educativo - si está disponible
        if self.available_columns['nivel'] and self.available_columns['nivel'] in self.label_encoders:
            col_nivel = self.available_columns['nivel']
            print("\n3. Nivel educativo deseado:")
            niveles_disponibles = list(self.label_encoders[col_nivel].classes_)
            for i, nivel in enumerate(niveles_disponibles, 1):
                print(f"   {i}. {nivel}")
            try:
                nivel_idx = int(input(f"Selecciona una opción (1-{len(niveles_disponibles)}): ")) - 1
                perfil_alumno['nivel'] = niveles_disponibles[nivel_idx]
            except:
                perfil_alumno['nivel'] = niveles_disponibles[0]
        
        # Familia profesional - si está disponible
        if self.available_columns['familia'] and self.available_columns['familia'] in self.label_encoders:
            col_familia = self.available_columns['familia']
            print("\n4. Área de interés principal:")
            familias = sorted(self.label_encoders[col_familia].classes_)
            # Excluir 'TOTAL' si existe
            familias = [f for f in familias if f != 'TOTAL']
            
            # Mostrar solo las primeras 20 si hay muchas
            familias_mostrar = familias[:20] if len(familias) > 20 else familias
            for i, familia in enumerate(familias_mostrar, 1):
                print(f"   {i}. {familia}")
            if len(familias) > 20:
                print(f"   ... y {len(familias) - 20} más")
            try:
                familia_idx = int(input(f"Selecciona una opción (1-{len(familias_mostrar)}): ")) - 1
                perfil_alumno['familia'] = familias_mostrar[familia_idx]
            except:
                perfil_alumno['familia'] = familias[0] if familias else 'Sin especificar'
        
        # Información adicional
        print("\n5. Rendimiento académico previo:")
        print("   1. Bajo (< 6)")
        print("   2. Medio (6-7)")
        print("   3. Alto (7-8.5)")
        print("   4. Excelente (> 8.5)")
        try:
            rendimiento = int(input("Selecciona una opción (1-4): "))
            perfil_alumno['rendimiento_previo'] = rendimiento
        except:
            perfil_alumno['rendimiento_previo'] = 2
        
        # Objetivos
        print("\n6. Objetivo profesional principal:")
        print("   1. Incorporación inmediata al mercado laboral")
        print("   2. Continuar estudios superiores")
        print("   3. Emprendimiento")
        print("   4. Especialización técnica")
        try:
            objetivo = int(input("Selecciona una opción (1-4): "))
            perfil_alumno['objetivo_profesional'] = objetivo
        except:
            perfil_alumno['objetivo_profesional'] = 1
        
        perfil_alumno['fecha_consulta'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        return perfil_alumno
    
    def generar_recomendaciones(self, perfil_alumno):
        """Genera recomendaciones basadas en el perfil del alumno"""
        print("\n=== Generando recomendaciones personalizadas ===")
        
        # Preparar features del alumno
        features_alumno = []
        
        # Mapear perfil a features disponibles
        for col in self.feature_columns:
            col_original = col.replace('_encoded', '')
            valor_default = 0
            
            # Buscar el tipo de columna
            tipo_columna = None
            for tipo, nombre_col in self.available_columns.items():
                if nombre_col == col_original:
                    tipo_columna = tipo
                    break
            
            if tipo_columna and tipo_columna in perfil_alumno:
                valor = perfil_alumno[tipo_columna]
                if valor in self.label_encoders[col_original].classes_:
                    features_alumno.append(self.label_encoders[col_original].transform([valor])[0])
                else:
                    features_alumno.append(valor_default)
            else:
                features_alumno.append(valor_default)
        
        # Escalar features
        features_alumno_scaled = self.scaler.transform([features_alumno])
        
        # 1. Predicción KNN
        probabilidades = self.knn_model.predict_proba(features_alumno_scaled)[0]
        prediccion_exito = self.knn_model.predict(features_alumno_scaled)[0]
        
        # 2. Cluster K-means
        cluster_alumno = self.kmeans_model.predict(features_alumno_scaled)[0]
        
        # 3. Análisis de vecinos similares
        distancias, indices = self.knn_model.kneighbors(features_alumno_scaled, n_neighbors=min(10, len(self.data_prepared)))
        vecinos_similares = self.data_prepared.iloc[indices[0]]
        
        # Generar recomendaciones basadas en vecinos exitosos
        recomendaciones = self._generar_recomendaciones_detalladas(
            perfil_alumno, vecinos_similares, cluster_alumno, probabilidades[1]
        )
        
        return recomendaciones
    
    def _generar_recomendaciones_detalladas(self, perfil, vecinos, cluster, prob_exito):
        """Genera recomendaciones detalladas basadas en el análisis"""
        recomendaciones = {
            'perfil': perfil,
            'estadisticas': {
                'probabilidad_exito': float(prob_exito),
                'cluster_asignado': int(cluster),
                'numero_vecinos_similares': len(vecinos)
            },
            'recomendaciones': []
        }
        
        # Analizar vecinos exitosos
        vecinos_exitosos = vecinos[vecinos['exito'] == 1]
        
        if len(vecinos_exitosos) > 0:
            # Analizar por familia profesional si está disponible
            if self.available_columns['familia']:
                col_familia = self.available_columns['familia']
                familias_exitosas = vecinos_exitosos[col_familia].value_counts()
                
                for familia, count in familias_exitosas.head(5).items():
                    if familia != 'TOTAL':
                        vecinos_familia = vecinos_exitosos[vecinos_exitosos[col_familia] == familia]
                        porcentaje_medio = vecinos_familia[self.porcentaje_col].mean()
                        
                        recomendacion = {
                            'tipo': 'Familia Profesional',
                            'valor': familia,
                            'porcentaje_aprobados_medio': float(porcentaje_medio),
                            'estudiantes_similares': int(count),
                            'confianza': float(count / len(vecinos_exitosos))
                        }
                        
                        # Añadir nivel si está disponible
                        if self.available_columns['nivel']:
                            col_nivel = self.available_columns['nivel']
                            nivel_mas_comun = vecinos_familia[col_nivel].mode()
                            if len(nivel_mas_comun) > 0:
                                recomendacion['nivel_recomendado'] = nivel_mas_comun.iloc[0]
                        
                        recomendaciones['recomendaciones'].append(recomendacion)
            
            # Recomendaciones generales basadas en el cluster
            datos_cluster = self.data_prepared[self.data_prepared['cluster'] == cluster]
            if len(datos_cluster) > 0:
                cluster_exitoso = datos_cluster['exito'].mean() > 0.5
                porcentaje_cluster = datos_cluster[self.porcentaje_col].mean()
                
                recomendaciones['estadisticas']['cluster_exitoso'] = bool(cluster_exitoso)
                recomendaciones['estadisticas']['porcentaje_medio_cluster'] = float(porcentaje_cluster)
        
        # Añadir recomendaciones basadas en el rendimiento previo
        if perfil.get('rendimiento_previo', 2) >= 3:
            recomendaciones['recomendaciones_adicionales'] = [
                "Tu rendimiento académico previo es bueno, considera ciclos de grado superior",
                "Podrías optar por familias profesionales más exigentes técnicamente"
            ]
        else:
            recomendaciones['recomendaciones_adicionales'] = [
                "Considera comenzar con ciclos de grado medio",
                "Busca apoyo adicional en las materias más difíciles"
            ]
        
        return recomendaciones
    
    def mostrar_recomendaciones(self, recomendaciones):
        """Muestra las recomendaciones de forma clara y estructurada"""
        print("\n" + "="*60)
        print("RECOMENDACIONES PERSONALIZADAS")
        print("="*60)
        
        # Mostrar perfil
        perfil = recomendaciones['perfil']
        print("\nTu perfil:")
        for key, value in perfil.items():
            if key != 'fecha_consulta':
                print(f"  • {key}: {value}")
        
        # Estadísticas
        stats = recomendaciones['estadisticas']
        print(f"\nProbabilidad de éxito: {stats['probabilidad_exito']*100:.1f}%")
        print(f"Cluster asignado: {stats['cluster_asignado']}")
        print(f"Estudiantes similares encontrados: {stats['numero_vecinos_similares']}")
        
        # Recomendaciones principales
        print("\n" + "-"*40)
        print("RECOMENDACIONES PRINCIPALES")
        print("-"*40)
        
        for i, rec in enumerate(recomendaciones['recomendaciones'], 1):
            print(f"\n{i}. {rec['valor']}")
            print(f"   Porcentaje de aprobados: {rec['porcentaje_aprobados_medio']:.1f}%")
            print(f"   Estudiantes similares exitosos: {rec['estudiantes_similares']}")
            print(f"   Nivel de confianza: {rec['confianza']*100:.1f}%")
            if 'nivel_recomendado' in rec:
                print(f"   Nivel recomendado: {rec['nivel_recomendado']}")
        
        # Recomendaciones adicionales
        if 'recomendaciones_adicionales' in recomendaciones:
            print("\n" + "-"*40)
            print("CONSEJOS ADICIONALES")
            print("-"*40)
            for consejo in recomendaciones['recomendaciones_adicionales']:
                print(f"• {consejo}")
        
        print("\n" + "="*60)
    
    def guardar_recomendaciones(self, recomendaciones):
        """Guarda las recomendaciones en archivos"""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        # Guardar JSON
        filename_json = f"{resultados_dir}/recomendaciones_{timestamp}.json"
        with open(filename_json, 'w', encoding='utf-8') as f:
            json.dump(recomendaciones, f, ensure_ascii=False, indent=2)
        
        # Guardar resumen CSV
        if recomendaciones['recomendaciones']:
            df_recomendaciones = pd.DataFrame(recomendaciones['recomendaciones'])
            filename_csv = f"{resultados_dir}/resumen_recomendaciones_{timestamp}.csv"
            df_recomendaciones.to_csv(filename_csv, index=False, encoding='utf-8')
            print(f"✓ Resumen guardado en: {filename_csv}")
        
        print(f"✓ Recomendaciones guardadas en: {filename_json}")
        
        return filename_json

# Función principal para ejecutar el sistema
def ejecutar_sistema_recomendacion():
    """Ejecuta el sistema completo de recomendaciones"""
    print("\n=== INICIANDO SISTEMA DE RECOMENDACIÓN ===")
    
    # Inicializar sistema
    sistema = SistemaRecomendacionAdaptativo()
    
    # Preparar datos
    try:
        X, y = sistema.preparar_datos(porcentajes, ciclos)
        print("✓ Datos preparados exitosamente")
    except Exception as e:
        print(f"Error preparando datos: {e}")
        return
    
    # Entrenar modelos
    try:
        accuracy, clusters = sistema.entrenar_modelos(X, y)
        print(f"✓ Modelos entrenados - Accuracy KNN: {accuracy:.3f}")
    except Exception as e:
        print(f"Error entrenando modelos: {e}")
        return
    
    # Interfaz interactiva
    while True:
        print("\n¿Qué deseas hacer?")
        print("1. Obtener recomendaciones personalizadas")
        print("2. Ver estadísticas generales")
        print("3. Salir")
        
        try:
            opcion = input("\nSelecciona una opción (1-3): ")
        except KeyboardInterrupt:
            print("\n\nSaliendo del sistema...")
            break
        
        if opcion == "1":
            try:
                # Obtener perfil del alumno
                perfil = sistema.crear_formulario_entrada()
                
                # Generar recomendaciones
                recomendaciones = sistema.generar_recomendaciones(perfil)
                
                # Mostrar recomendaciones
                sistema.mostrar_recomendaciones(recomendaciones)
                
                # Guardar recomendaciones
                sistema.guardar_recomendaciones(recomendaciones)
                
            except Exception as e:
                print(f"Error generando recomendaciones: {e}")
        
        elif opcion == "2":
            mostrar_estadisticas_generales(sistema)
        
        elif opcion == "3":
            print("\n¡Gracias por usar el sistema de recomendación!")
            break
        
        else:
            print("\nOpción no válida. Por favor, selecciona 1, 2 o 3.")

def mostrar_estadisticas_generales(sistema):
    """Muestra estadísticas generales del sistema"""
    print("\n=== ESTADÍSTICAS GENERALES ===")
    
    data = sistema.data_prepared
    
    # Estadísticas básicas
    print(f"\nTotal de registros: {len(data)}")
    print(f"Tasa de éxito general: {data['exito'].mean()*100:.1f}%")
    print(f"Porcentaje medio de aprobados: {data[sistema.porcentaje_col].mean():.1f}%")
    
    # Por familia profesional si está disponible
    if sistema.available_columns['familia']:
        col_familia = sistema.available_columns['familia']
        print(f"\nEstadísticas por {col_familia}:")
        stats_familia = data.groupby(col_familia).agg({
            sistema.porcentaje_col: ['mean', 'count'],
            'exito': 'mean'
        }).round(2)
        
        # Ordenar por porcentaje de aprobados
        stats_familia = stats_familia.sort_values((sistema.porcentaje_col, 'mean'), ascending=False)
        print(stats_familia.head(10))
    
    # Por nivel educativo si está disponible
    if sistema.available_columns['nivel']:
        col_nivel = sistema.available_columns['nivel']
        print(f"\nEstadísticas por {col_nivel}:")
        stats_nivel = data.groupby(col_nivel).agg({
            sistema.porcentaje_col: ['mean', 'count'],
            'exito': 'mean'
        }).round(2)
        print(stats_nivel)
    
    # Por cluster
    print("\nEstadísticas por cluster:")
    stats_cluster = data.groupby('cluster').agg({
        sistema.porcentaje_col: ['mean', 'count'],
        'exito': 'mean'
    }).round(2)
    print(stats_cluster)

# Validación del sistema
def validar_sistema(sistema):
    """Valida el sistema con casos de prueba"""
    print("\n=== VALIDACIÓN DEL SISTEMA ===")
    
    casos_prueba = []
    
    # Crear casos de prueba basados en las columnas disponibles
    if sistema.available_columns['familia'] and sistema.label_encoders[sistema.available_columns['familia']].classes_.size > 0:
        familias = list(sistema.label_encoders[sistema.available_columns['familia']].classes_)[:3]
        for i, familia in enumerate(familias):
            caso = {
                'nombre': f'Caso {i+1}',
                'familia': familia,
                'rendimiento_previo': i + 2,
                'objetivo_profesional': i + 1
            }
            
            # Añadir otras características si están disponibles
            if sistema.available_columns['sexo']:
                sexos = list(sistema.label_encoders[sistema.available_columns['sexo']].classes_)
                caso['sexo'] = sexos[i % len(sexos)]
            
            if sistema.available_columns['nivel']:
                niveles = list(sistema.label_encoders[sistema.available_columns['nivel']].classes_)
                caso['nivel'] = niveles[i % len(niveles)]
            
            casos_prueba.append(caso)
    
    resultados = []
    for caso in casos_prueba:
        try:
            recomendaciones = sistema.generar_recomendaciones(caso)
            resultado = {
                'caso': caso['nombre'],
                'probabilidad_exito': recomendaciones['estadisticas']['probabilidad_exito'],
                'cluster': recomendaciones['estadisticas']['cluster_asignado'],
                'num_recomendaciones': len(recomendaciones['recomendaciones'])
            }
            resultados.append(resultado)
            print(f"✓ {caso['nombre']}: Éxito {resultado['probabilidad_exito']*100:.1f}%")
        except Exception as e:
            print(f"✗ Error en {caso['nombre']}: {e}")
    
    return resultados

# Ejecutar el sistema
if __name__ == "__main__":
    # Intentar ejecutar el sistema
    try:
        ejecutar_sistema_recomendacion()
    except Exception as e:
        print(f"Error crítico: {e}")
        print("\nIntentando modo de demostración...")
        
        # Modo de demostración con validación
        try:
            sistema = SistemaRecomendacionAdaptativo()
            X, y = sistema.preparar_datos(porcentajes, ciclos)
            accuracy, clusters = sistema.entrenar_modelos(X, y)
            resultados_validacion = validar_sistema(sistema)
            print("\n✓ Sistema validado correctamente")
        except Exception as e2:
            print(f"Error en modo de demostración: {e2}")

✓ Carpeta existente: ./resultados_recomendaciones
✓ Datos cargados:
  - Porcentajes: (4212, 5)
  - Ciclos: (47250, 5)

Columnas en porcentajes:
['Sexo', 'Comunidad autónoma', 'Familia profesional', 'Porcentajes total de módulos aprobados', 'nivel_educativo']

Primeras filas de porcentajes:
          Sexo Comunidad autónoma               Familia profesional  \
0  AMBOS SEXOS          Andalucía  ACTIVIDADES FÍSICAS Y DEPORTIVAS   
1  AMBOS SEXOS          Andalucía          ADMINISTRACIÓN Y GESTIÓN   
2  AMBOS SEXOS          Andalucía                           AGRARIA   
3  AMBOS SEXOS          Andalucía                    ARTES GRÁFICAS   
4  AMBOS SEXOS          Andalucía                ARTES Y ARTESANÍAS   

   Porcentajes total de módulos aprobados nivel_educativo  
0                                    61.5          BASICO  
1                                    75.8          BASICO  
2                                    73.5          BASICO  
3                                    76.0 